### 1. Install and Import Dependancies

In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import requests
from bs4 import BeautifulSoup
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification

### 2. Initiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### 3. Encode and Calculate sentiment

In [4]:
# Example
tokens = tokenizer.encode('I hated this, absolutely worst', return_tensors='pt') #pytorch

In [5]:
tokens

tensor([[  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 43060,   102]])

In [6]:
tokens[0]

tensor([  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 43060,   102])

In [7]:
tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutely worst [SEP]'

In [8]:
result = model(tokens)

In [9]:
result 
# The output from the model is a one-hot encoded list of scores
# The position with highest score represents the sentiment rating

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.9280,  1.8564, -0.7541, -2.9172, -2.2860]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [10]:
result.logits

tensor([[ 4.9280,  1.8564, -0.7541, -2.9172, -2.2860]],
       grad_fn=<AddmmBackward0>)

In [11]:
torch.argmax(result.logits)

tensor(0)

In [12]:
int(torch.argmax(result.logits))+1 # '1' indicates negative sentiment

1

### Examples

In [13]:
tokens = tokenizer.encode('This is amazing, I loved it. GREAT !', return_tensors='pt') #pytorch
result = model(tokens)
int(torch.argmax(result.logits))+1

5

In [14]:
tokens = tokenizer.encode('Man, it was okayy', return_tensors='pt') #pytorch
result = model(tokens)
int(torch.argmax(result.logits))+1

3

In [15]:
# Lower the score - worst the sentiment 
# Higher the score - better the sentiment

### 4. Collect Reviews

In [16]:
r = requests.get('https://www.dineout.co.in/pune/mainland-china-senapati-bapat-road-west-pune-3777/review?revpage=6')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('more')
results = soup.find_all('span',{'class':regex})
reviews = [result.text for result in results]

In [17]:
r

<Response [200]>

In [18]:
r.text

'<!DOCTYPE html><html lang="en"><head><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no"><link rel="manifest" href="/manifest.json"><style type="text/css">\n            @font-face {\n                font-family: \'dineicon\';\n                src:  url(\'/fonts/dineicon.eot\');\n                src:  url(\'/fonts/dineicon.eot#iefix\') format(\'embedded-opentype\'),\n                url(\'/fonts/dineicon.ttf\') format(\'truetype\'),\n                url(\'/fonts/dineicon.woff\') format(\'woff\'),\n                url(\'/fonts/dineicon.svg#dineicon\') format(\'svg\');\n                font-weight: normal;\n\t\t\t\tfont-style: normal;\n\t\t\t\tfont-display: swap;\n            }\n            .hide {\n                display: none !important;\n            }\n            .async-hide{\n                opacity: inherit !important;\n            }\n            iframe

In [19]:
results

[<span class="more">Set menu option is not now???????</span>,
 <span class="more">food was very tasty, ambience was excellent. I really enjoyed it</span>,
 <span class="more">food quality detoriated to a large extent.</span>,
 <span class="more">As per dineout deal, 15% discount was not given though????????. 
 But as usual enjoyed the meal????</span>,
 <span class="more">Crab dishes could have been better- few were bitter</span>,
 <span class="more">Patjatic food options and quality quantity</span>,
 <span class="more">food was delicious. portions were good too.</span>,
 <span class="more">The food taste is ok ok.  The wow factor has gone out from the items.</span>,
 <span class="more">one of the best places in Pune to have a dinner with Family.</span>,
 <span class="more">the best dishes to try here are Schezwan baby corn, crispy corn chilli pepper, chilli garlic noodles and hot garlic panfried noodles.Also their sizzling brownie is really great .very good service.</span>,
 <span clas

In [20]:
results[0]

<span class="more">Set menu option is not now???????</span>

In [21]:
results[0].text

'Set menu option is not now???????'

In [22]:
reviews

['Set menu option is not now???????',
 'food was very tasty, ambience was excellent. I really enjoyed it',
 'food quality detoriated to a large extent.',
 'As per dineout deal, 15% discount was not given though????????. \nBut as usual enjoyed the meal????',
 'Crab dishes could have been better- few were bitter',
 'Patjatic food options and quality quantity',
 'food was delicious. portions were good too.',
 'The food taste is ok ok.  The wow factor has gone out from the items.',
 'one of the best places in Pune to have a dinner with Family.',
 'the best dishes to try here are Schezwan baby corn, crispy corn chilli pepper, chilli garlic noodles and hot garlic panfried noodles.Also their sizzling brownie is really great .very good service.',
 "my favorite ice-cream wasn't available...rest was good",
 'excellent food and service',
 'excellent food & Service.',
 'great customer service with tasty food. unbeatable!!!',
 'Great value for money with 50% off on food.  Jumbo Prawns could’ve been

In [23]:
reviews[0]

'Set menu option is not now???????'

### 5. Load Reviews into Dataframe

In [24]:
import pandas as pd
import numpy as np

In [25]:
df = pd.DataFrame(np.array(reviews),columns=['review'])

In [26]:
df

,review
0,Set menu option is not now???????
1,"food was very tasty, ambience was excellent. I..."
2,food quality detoriated to a large extent.
3,"As per dineout deal, 15% discount was not give..."
4,Crab dishes could have been better- few were b...
5,Patjatic food options and quality quantity
6,food was delicious. portions were good too.
7,The food taste is ok ok. The wow factor has g...
8,one of the best places in Pune to have a dinne...
9,the best dishes to try here are Schezwan baby ...


In [27]:
df['review'].iloc[0]

'Set menu option is not now???????'

In [28]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1    

In [29]:
print(df['review'].iloc[0])
sentiment_score(df['review'].iloc[0])

Set menu option is not now???????


1

In [30]:
print(df['review'].iloc[3])
sentiment_score(df['review'].iloc[3])

As per dineout deal, 15% discount was not given though????????. 
But as usual enjoyed the meal????


3

In [31]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x)) # limitation of tokens - x[:512]

In [32]:
pd.options.display.max_colwidth = 200
df

,review,sentiment
0,Set menu option is not now???????,1
1,"food was very tasty, ambience was excellent. I really enjoyed it",5
2,food quality detoriated to a large extent.,2
3,"As per dineout deal, 15% discount was not given though????????. \nBut as usual enjoyed the meal????",3
4,Crab dishes could have been better- few were bitter,3
5,Patjatic food options and quality quantity,5
6,food was delicious. portions were good too.,4
7,The food taste is ok ok. The wow factor has gone out from the items.,3
8,one of the best places in Pune to have a dinner with Family.,5
9,"the best dishes to try here are Schezwan baby corn, crispy corn chilli pepper, chilli garlic noodles and hot garlic panfried noodles.Also their sizzling brownie is really great .very good service.",5
